# Data Engineering Test
# Solvex - 2024

## Resuelto por Santiago Taracena Puga

### Ejercicio 3: Integración de Pandas y Spark con datos de películas y críticas

Supongamos que tienes dos conjuntos de datos: uno en Pandas y otro en Spark. El conjunto de datos de Pandas es una tabla llamada "datos_peliculas" con información sobre películas:

```
ID,Título,Año
1,Película1,2020
2,Película2,2019
3,Película3,2021
4,Película4,2018
```

El conjunto de datos de Spark es un DataFrame llamado "criticas" con información sobre las críticas de películas:

```
PeliculaID,Critico,Puntuacion
1,Critico1,4.5
2,Critico2,3.8
3,Critico1,4.2
4,Critico3,4.7
```

Combina estos dos conjuntos de datos para obtener una tabla que muestre el título de la película, el año de lanzamiento y la puntuación promedio de las críticas. Asegúrate de utilizar tanto Pandas como Spark en el proceso de integración.

### Solución

Para la resolución del presente ejercicio, es necesario en primer lugar importar y crear el primer dataset correspondiente a Pandas. Podemos importar Pandas como lo hemos hecho anteriormente, y también utilizar la clase `DataFrame` con el objetivo de crear el primer dataset mencionado.

In [ ]:
# Instrucción para importar Pandas
import pandas as pd

In [ ]:
# Creación del dataset de películas
movies_data = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "title": ["Película1", "Película2", "Película3", "Película4"],
    "year": [2020, 2019, 2021, 2018],
})
movies_data

,id,title,year
0,1,Película1,2020
1,2,Película2,2019
2,3,Película3,2021
3,4,Película4,2018


Con el primer dataset de Pandas finalizado, lo siguiente es crear el dataset de Spark. Tenemos qué importar Spark como lo hemos realizado anteriormente, y continuar el procedimiento de instanciarlo y crear un DataFrame que en lugar de Pandas sea de Spark. Podemos crear un DataFrame de Spark con la función `createDataFrame`, pasando como argumentos la data y las columnas.

In [ ]:
# Instrucción para importar Spark
from pyspark.sql import SparkSession

In [ ]:
# Instancia para trabajar con Spark
spark = SparkSession.builder.appName("PandasSpark").getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
# Creación del dataset de Spark
data = [
    (0, "Crítico1", 4.5),
    (1, "Crítico2", 3.8),
    (2, "Crítico1", 4.2),
    (3, "Crítico3", 4.7),
]
reviews = spark.createDataFrame(data, ["movie_id", "reviewer", "score"])
reviews.show()

+--------+--------+-----+
|movie_id|reviewer|score|
+--------+--------+-----+
|       0|Crítico1|  4.5|
|       1|Crítico2|  3.8|
|       2|Crítico1|  4.2|
|       3|Crítico3|  4.7|
+--------+--------+-----+



Con los dos datasets, podemos hacer una integración entre los mismos dos creando un DataFrame de Spark con la función `createDataFrame` anterior. Spark tiene una particularidad muy útil, especialmente con esta función, y es que si pasamos un DataFrame de Pandas, este mismo puede ser casteado a uno de Spark sin ningún problema.

In [ ]:
# Casteo del DataFrame de Pandas a Spark
movies_data = spark.createDataFrame(movies_data)
movies_data.show()

+---+---------+----+
| id|    title|year|
+---+---------+----+
|  1|Película1|2020|
|  2|Película2|2019|
|  3|Película3|2021|
|  4|Película4|2018|
+---+---------+----+



Con ambos datasets en el formato que necesitamos, podemos proceder a ejecutar un join para tener los datos de las dos tablas unidos para obtener el promedio mencionado. La función `join` nos permite realizar este join, identificando la tabla a unir, la condición del join y qué tipo de join es. Básicamente estamos haciendo un ```SELECT * FROM movies INNER JOIN reviews ON movies.id = reviews.movie_id```.

In [ ]:
# Join entre las dos tablas creadas
joined_data = movies_data.join(reviews, movies_data.id == reviews.movie_id, "inner")
joined_data.show()

+---+---------+----+--------+--------+-----+
| id|    title|year|movie_id|reviewer|score|
+---+---------+----+--------+--------+-----+
|  1|Película1|2020|       1|Crítico2|  3.8|
|  2|Película2|2019|       2|Crítico1|  4.2|
|  3|Película3|2021|       3|Crítico3|  4.7|
+---+---------+----+--------+--------+-----+



Finalmente, con los dos datasets unidos, únicamente hace falta obtener el promedio de cada película agrupada por título y año. Esto podemos hacerlo con la función `groupBy`, y posteriormente utilizando la función de agregación `avg` para obtener el promedio solicitado.

In [ ]:
# Función avg de Spark
from pyspark.sql.functions import avg

In [ ]:
# Tabla resultante de obtener los promedios
result = joined_data.groupBy("title", "year").agg(avg("score").alias("avg_score"))
result.show()

+---------+----+---------+
|    title|year|avg_score|
+---------+----+---------+
|Película1|2020|      3.8|
|Película2|2019|      4.2|
|Película3|2021|      4.7|
+---------+----+---------+

